In [6]:
# Graph the array as gífs
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter

In [7]:
x = np.load("x_solved.npy")

shape = x.shape
print("Shape of x: ", shape)

Shape of x:  (128, 128, 128)


In [ ]:
# ---- helpers ----
def slice_window(z, lo=1, hi=99):
    vmin, vmax = np.percentile(z, (lo, hi))
    if vmax <= vmin:
        vmax = vmin + 1e-9
    return float(vmin), float(vmax)

def make_slice_anim(x, axis, outfile, fps=5, lo=1, hi=99, *,
                    scale=2.0, dpi=200, interpolation="nearest"):
    """
    axis: 0, 1, or 2  -> vary i, j, or k respectively
    scale: multiplies figure size (≈ pixel dimensions of the GIF)
    dpi: figure DPI (controls absolute pixel count with figsize)
    interpolation: 'nearest' (crisp) or 'bilinear' (smoother)
    """
    nframes = x.shape[axis]
    lab = ["i", "j", "k"][axis]

    if axis == 0:
        get = lambda idx: x[idx, :, :]
    elif axis == 1:
        get = lambda idx: x[:, idx, :]
    else:
        get = lambda idx: x[:, :, idx]

    z0 = get(0)
    vmin0, vmax0 = slice_window(z0, lo=lo, hi=hi)

    # Base square figure; scale > 1 upsizes the output
    base_size = 5.0  # inches; arbitrary, only relative to dpi
    fig, ax = plt.subplots(figsize=(base_size*scale, base_size*scale), dpi=dpi)
    ax.set_axis_off()

    im = ax.imshow(z0, cmap="bone", vmin=vmin0, vmax=vmax0,
                   animated=False, interpolation=interpolation)
    cb = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cb.set_label("Intensity")

    txt = ax.text(5, 5, f"{lab} = 0", color="w", fontsize=10*scale)

    def update(idx):
        z = get(idx)
        vmin, vmax = slice_window(z, lo=lo, hi=hi)
        im.set_data(z)
        im.set_clim(vmin=vmin, vmax=vmax)
        cb.update_normal(im)
        txt.set_text(f"{lab} = {idx}")
        return im, txt, cb.ax

    ani = FuncAnimation(fig, update, frames=nframes, interval=50, blit=False, repeat=True)
    ani.save(outfile, writer=PillowWriter(fps=fps))
    plt.close(fig)

# ---- build all three animations (2× upscaled) ----
make_slice_anim(x, axis=0, outfile="slices_1.gif", fps=5, scale=2.0, interpolation="nearest")
make_slice_anim(x, axis=1, outfile="slices_2.gif", fps=5, scale=2.0, interpolation="nearest")
make_slice_anim(x, axis=2, outfile="slices_3.gif", fps=5, scale=2.0, interpolation="nearest")
